In [ ]:
import cv2
import numpy as np

In [ ]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 10
    # lower_hsv = np.array([60 - sensitivity, 100, 50])
    # upper_hsv = np.array([60 + sensitivity, 255, 255])
    lower_hsv = np.array([35 - sensitivity, 43, 46])
    upper_hsv = np.array([77 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))  # 椭圆结构
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  # 进行闭运算（将物体合并）

    #  通过腐蚀，膨胀将一些小的噪音去除
    kernel = np.ones((3, 3), np.int8)
    mask = cv2.erode(mask, kernel=kernel, iterations=2)
    mask = cv2.dilate(mask, kernel=kernel, iterations=2)

    return mask


def segment_plant(image):
    mask = create_mask_for_plant(image)
    segment_plant = cv2.bitwise_and(image, image, mask=mask)
    return segment_plant


def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [ ]:
classes = 12
scale = 70
seed = 7

def creat_model_kaggle():
    from keras.models import Sequential
    from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense

    model = Sequential()

    model.add(
        Conv2D(filters=64, kernel_size=(5, 5), input_shape=(3, scale, scale), activation='relu', dim_ordering='th'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu', dim_ordering='th'))
    model.add(MaxPooling2D((2, 2), dim_ordering='th'))
    model.add(BatchNormalization(axis=3))
    model.add(Dropout(0.1))

    model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu', dim_ordering='th'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu', dim_ordering='th'))
    model.add(MaxPooling2D((2, 2), dim_ordering='th'))
    model.add(BatchNormalization(axis=3))
    model.add(Dropout(0.1))

    model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu', dim_ordering='th'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu', dim_ordering='th'))
    model.add(MaxPooling2D((2, 2), dim_ordering='th'))
    model.add(BatchNormalization(axis=3))
    model.add(Dropout(0.1))

    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # model.summary()
    return model


In [ ]:
def load_train(train_path):
    # 该函数读取数据集，并且对数据集进行处理，以便于之后放入keras的网络中进行训练
    import os
    import glob
    import cv2
    import numpy as np

    x_train = []
    x_train_class = []
    x_train_index = []
    print('Reading train images ... ')

    cate = []
    for x in os.listdir(train_path):
        if os.path.isdir(train_path + x):
            cate.append(train_path + x)

    for index, floder in enumerate(cate):
        for im in glob.glob(floder + '/*.png'):
            image = cv2.resize(cv2.imread(im), (scale, scale))
            segment_plants = segment_plant(image=image)
            x_train.append(segment_plants)
            x_train_class.append(os.path.basename(im))
            x_train_index.append(index)

    return np.asarray(x_train), x_train_class, np.asarray(x_train_index)


def read_normalize_train_data(train_path):
    from keras.utils import np_utils
    import numpy as np
    x_train, x_train_class, x_train_index = load_train(train_path=train_path)

    print('Reshape:np.transpose ... ')
    x_train = x_train.transpose((0, 3, 1, 2))  # transpose之后的(train个数，channels,weight,height)
    x_train = x_train.astype('float32')
    x_train = x_train / 255
    x_train_index = np_utils.to_categorical(x_train_index, len(np.unique(x_train_index)))

    print('Train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')

    return x_train, x_train_class, x_train_index


In [ ]:

def read_normalize_test_data(img_path):
    import cv2
    import numpy as np
    from keras.preprocessing.image import img_to_array

    plant = segment_plant(image=cv2.resize(cv2.imread(img_path), (scale, scale)))
    # cv2.imshow('',plant)
    # cv2.waitKey(0)
    plant = img_to_array(plant)
    plant = [plant]
    plant = np.array(plant, dtype=np.uint8)
    plant = plant.transpose((0, 3, 1, 2))
    plant = plant.astype('float32')
    plant = plant / 255
    return plant

In [ ]:
def run_cross_validation_create_models(train_path, nfolds=10):
    from sklearn.cross_validation import KFold
    from keras.callbacks import EarlyStopping
    from sklearn.metrics import log_loss
    import matplotlib.pyplot as plt

    batch_size = 32
    nb_epoch = 50  # 训练次数
    random_state = seed  # 随机数种子的数量
    x_train, x_train_class, x_train_index = read_normalize_train_data(train_path)

    kf = KFold(len(x_train), n_folds=nfolds, shuffle=True, random_state=random_state)
    num_fold = 0
    sum_loss = 0

    for train_index, test_index in kf:
        X_train = x_train[train_index]
        X_val = x_train[test_index]
        Y_train = x_train_index[train_index]
        Y_val = x_train_index[test_index]

        num_fold += 1
        print('*' * 50)

        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train))
        print('Split valid: ', len(X_val))

        model = creat_model_kaggle()
        # model = multi_gpu_model(model=model, gpus=2)

        # 提前停止训练:patience:能够容忍多少个epoch内都没有improvement,verbose:是否打印出提前停止训练的相关信息
        callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=0)]

        result = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=0, callbacks=callbacks,
                           validation_data=(X_val, Y_val), shuffle=True)

        # model.save('models/plant_segment_' + str(num_fold) + '.h5')

        predictions_val = model.predict(X_val.astype('float32'), batch_size=batch_size, verbose=2)
        logloss = log_loss(Y_val, predictions_val)  # 对数损失函数
        loss, acc = model.evaluate(X_train, Y_train, verbose=0)

        sum_loss += logloss * len(test_index)
        print('Score log_loss:{}'.format(logloss))
        print('Score loss:{}'.format(acc))
        print('Score accuracy:{}'.format(acc))

        plt.figure()
        plt.plot(result.epoch, result.history['acc'], label="acc")
        plt.plot(result.epoch, result.history['val_acc'], label="val_acc")
        plt.scatter(result.epoch, result.history['acc'], marker='*')
        plt.scatter(result.epoch, result.history['val_acc'])
        plt.legend(loc='under right')
        # plt.savefig('evaluation/plant_segment_accuracy_' + str(num_fold) + '.jpg')

        plt.figure()
        plt.plot(result.epoch, result.history['loss'], label="loss")
        plt.plot(result.epoch, result.history['val_loss'], label="val_loss")
        plt.scatter(result.epoch, result.history['loss'], marker='*')
        plt.scatter(result.epoch, result.history['val_loss'])
        plt.legend(loc='upper right')
        # plt.savefig('evaluation/plant_segment_loss_' + str(num_fold) + '.jpg')

    avg_loss = sum_loss / len(x_train)
    print("Log_Loss_Avg train independent: ", avg_loss)


if __name__ == '__main__':
    import time
    import tensorflow as tf

    # import keras.backend.tensorflow_backend as KTF
    #
    # KTF.set_session(tf.Session(config=tf.ConfigProto(device_count={'gpu': 0})))
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    
    run_cross_validation_create_models(train_path='../input/train/')
